In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)


/home/trec/anaconda3/envs/PyBulletRL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lstm = nn.LSTM(3,3) # Input dim is 3, output dim is 3
inputs = [torch.randn(1,3) for i in range(5)] # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1,1,3), torch.randn(1,1,3)) # h_0, c_0

for input in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(input.view(1,1,-1), hidden) # input.view(1,1,-1) is to reshape the input to (1,1,3)
    
print(out)
print(hidden)


#alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout the sequence.
# the second is just the most recent hidden state (compare the last slice of "out" with "hidden" below, they are the same)
# the reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropogate, by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs),1,-1)
hidden = (torch.randn(1,1,3), torch.randn(1,1,3)) # clean out hidden state
out, hidden = lstm(inputs, hidden)
print("Next")
print(out)
print(hidden)


tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3600,  0.0893,  0.0215]]], grad_fn=<StackBackward0>), tensor([[[-1.1298,  0.4467,  0.0254]]], grad_fn=<StackBackward0>))
Next
tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward0>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward0>))


In [5]:
inputs = [torch.randn(1,3) for i in range(5)]
input[0].view(1,1,-1).shape


torch.Size([1, 1, 3])

Example: An LSTM for Part-of-Speech Tagging
In this section, we will use an LSTM to get part of speech tags. We will not use Viterbi or Forward-Backward or anything like that, but as a (challenging) exercise to the reader, think about how Viterbi could be used after you have seen what is going on. In this example, we also refer to embeddings. If you are unfamiliar with embeddings, you can read up about them
https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html?highlight=lstm

In [6]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

training_data = [("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),("Everybody read that book".split(), ["NN", "V", "DET", "NN"])]
    #Tags are: DET - determiner, NN - noun, V - verb
    #For example, the word "the" is a determiner, and the word "dog" is a noun
word_to_ix = {}
#For each words-list and tag-list in the training data

for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix: #the word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)#assign it the next available unique index
print(word_to_ix)

tag_to_ix = {"DET": 0, "NN": 1, "V": 2}#Assign each tag a unique index

#These will usually be more like32 or 64 dimensional.
#We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
    

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [7]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size) -> None:
        super(LSTMTagger,self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim,hidden_dim)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, hidden_state = self.lstm(embeds.view(len(sentence),1,-1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [8]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(),lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0],word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
    
for epoch in range(300): # again, normally you would NOT do 300 epochs, it is toy data 
    for sentence, tags in training_data:
        #Step 1. Remember that Pytorch accumulates gradients.
        #We need to clear them out before each instance
        model.zero_grad()
        
        #Step 2. Get our inputs ready for the network, that is, turn them into
        #Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        #Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        
        #Step 4. Compute the loss, gradients, and update the parameters by
        #calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        
        
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j for word i.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-1.2343, -1.4184, -0.7617],
        [-1.2152, -1.3930, -0.7874],
        [-1.1891, -1.4521, -0.7734],
        [-1.2456, -1.4261, -0.7508],
        [-1.2492, -1.4088, -0.7576]])
tensor([[-0.0477, -4.7206, -3.2795],
        [-3.8209, -0.0369, -4.2453],
        [-3.0973, -4.7365, -0.0555],
        [-0.0403, -4.6258, -3.5173],
        [-3.8273, -0.0289, -4.9976]])
